# Visualizing Data by Segment of Footprint

In [1]:
import warnings
warnings.filterwarnings("ignore") # suppress warnings

In [2]:
import os
import sys
import h5py
import laspy
import pyproj
import datetime 
import numpy as np
import pandas as pd
import geopandas as gpd
import contextily as ctx  # For adding a basemap
from typing import Tuple
import matplotlib.cm as cm
import matplotlib.pyplot as plt
from scipy.spatial import cKDTree
from matplotlib.cm import get_cmap
import matplotlib.colors as mcolors
import matplotlib.patches as patches
from scipy.interpolate import griddata
from mpl_toolkits.mplot3d import Axes3D
from matplotlib.patches import Circle, Patch
from geoviews import opts, tile_sources as gvts
from shapely.geometry import Point, box, Polygon
from matplotlib.collections import PatchCollection
from sliderule import sliderule, icesat2, earthdata
from matplotlib.patches import Polygon as MplPolygon
from mpl_toolkits.mplot3d.art3d import Poly3DCollection

plt.style.use("~/geoscience/carbon_estimation/MNRAS.mplstyle")
%matplotlib inline

In [3]:
function_path = os.path.expanduser("~/geoscience/carbon_estimation/code/functions")
sys.path.append(function_path)

# Check the default PROJ data directory
print(pyproj.datadir.get_data_dir())

/bsuscratch/tnde/miniforge3/envs/veg_height/share/proj


## Loading ICESat2 Data

In [4]:
from helper_functions import load_icesat2_data
projected_crs = "epsg:6340" 
icesat2_file = "icesat2_ml2.geojson"
icesat2_dir = "/bsuhome/tnde/scratch/carbon_estimation/data/icesat2_data/ml_data/"
atl08 = icesat2_dir+f'{icesat2_file}'
atl08

'/bsuhome/tnde/scratch/carbon_estimation/data/icesat2_data/ml_data/icesat2_ml2.geojson'

In [5]:
%%time
icesat2_gdf_atl08 = load_icesat2_data(atl08, "ATL08", process = False)
icesat2_gdf_atl08 = icesat2_gdf_atl08[icesat2_gdf_atl08["gnd_ph_count"] != 0]
icesat2_gdf_atl08 = icesat2_gdf_atl08[icesat2_gdf_atl08["h_canopy"] <= 15]
icesat2_gdf_atl08 = icesat2_gdf_atl08.rename(columns={"elevation_NAVD88_icesat2_from_crs": "Elevation"})
print(f"Length of data: {len(icesat2_gdf_atl08)}")
print(icesat2_gdf_atl08.crs, "\n")
icesat2_gdf_atl08.crs

Length of data: 30537
EPSG:6340 

CPU times: user 2.59 s, sys: 47.2 ms, total: 2.64 s
Wall time: 2.68 s


<Projected CRS: EPSG:6340>
Name: NAD83(2011) / UTM zone 11N
Axis Info [cartesian]:
- E[east]: Easting (metre)
- N[north]: Northing (metre)
Area of Use:
- name: United States (USA) - between 120°W and 114°W onshore and offshore - California, Idaho, Nevada, Oregon, Washington.
- bounds: (-120.0, 30.88, -114.0, 49.01)
Coordinate Operation:
- name: UTM zone 11N
- method: Transverse Mercator
Datum: NAD83 (National Spatial Reference System 2011)
- Ellipsoid: GRS 1980
- Prime Meridian: Greenwich

In [6]:
# icesat2_gdf_atl08.columns

In [7]:
# Convert the 'date' column to datetime format and keep only the date part (YYYY-MM-DD)
icesat2_gdf_atl08["date"] = pd.to_datetime(icesat2_gdf_atl08["time"]).dt.strftime("%Y-%m-%d")
# Print the first few rows to verify
icesat2_gdf_atl08.head()

,time,x_atc,h_min_canopy,h_te_median,gt,spot,rgt,gnd_ph_count,h_mean_canopy,h_canopy,...,h_dif_ref,Longitude,Latitude,Easting,Northing,elevation_NAVD88_icesat2_from_pipe,Elevation,Date,date,geometry
0,2019-05-01 02:00:18.797,15261785.0,0.518799,1669.7802,10,1,501,21,1.062189,3.419067,...,-4.134033,-116.847629,43.208991,512378.580431,4.784034e+06,1686.422704,1686.967398,2019-05-01 02:00:18.797,2019-05-01,POINT (512378.58 4784034.098)
1,2019-05-01 02:00:18.808,15261861.0,0.620728,1673.5652,10,1,501,40,1.759356,5.692627,...,-7.536011,-116.847721,43.208309,512371.297938,4.783958e+06,1690.205964,1690.750670,2019-05-01 02:00:18.808,2019-05-01,POINT (512371.298 4783958.308)
3,2019-05-01 02:00:18.837,15262066.0,1.146484,1718.3307,10,1,501,7,4.224749,8.523682,...,0.887329,-116.847964,43.206469,512351.895277,4.783754e+06,1734.966712,1735.511452,2019-05-01 02:00:18.837,2019-05-01,POINT (512351.895 4783753.969)
4,2019-05-01 02:00:18.849,15262149.0,0.557983,1720.0553,10,1,501,21,1.534317,3.273193,...,0.887329,-116.848063,43.205731,512344.028471,4.783672e+06,1736.689424,1737.234177,2019-05-01 02:00:18.849,2019-05-01,POINT (512344.028 4783672.001)
5,2019-05-01 02:00:18.864,15262262.0,0.520020,1719.2477,10,1,501,42,1.283726,3.224731,...,-2.612915,-116.848200,43.204718,512333.050720,4.783559e+06,1735.879210,1736.423982,2019-05-01 02:00:18.864,2019-05-01,POINT (512333.051 4783559.417)


## Loading GEDI Data

In [8]:
drop_cols = ["shot_number", "Latitude_l2a", "Longitude_l2a", 
             "rh_85", "rh_90", "rh_98", "rh_99", "GEDI02_A_slope_ratio_utm", 
             "GEDI02_A_slope_rise_run", "cover_z_10", "cover_z_15", "cover_z_20", 
             "cover_z_25", "cover_z_27", "cover_z_28", "pai_z_10", "pai_z_15", 
             "pai_z_20", "pai_z_25", "pai_z_27", "pai_z_28", "pavd_z_10", 
             "pavd_z_15", "pavd_z_20", "pavd_z_25", "pavd_z_27", "pavd_z_28", "rh100", "geometry"]

In [9]:
target_col = "rh_100" # "rh_80"
gedi_dir = "/bsuhome/tnde/scratch/carbon_estimation/data/gedi_data/gedi_finder/ml_data/"
gedi_file = "gedi_ml2.geojson"
gedi_data = gpd.read_file(os.path.join(gedi_dir, gedi_file))
gedi_data = gedi_data.drop(drop_cols, axis = 1)
gedi_data = gedi_data[gedi_data["fhd_normal"] != -9999.000000]
gedi_data = gedi_data[gedi_data[target_col] <= 15]
gedi_data = gedi_data.rename(columns={"Easting_l2a": "Easting", "Northing_l2a": "Northing", "BEAM_l2a": "BEAM", "elevation_NAVD88_gedi_from_crs_l2a": "Elevation"})
gedi_data.head()

,date,BEAM,quality_flag,num_detectedmodes,rh_80,rh_95,rh_100,sensitivity,solar_elevation,Easting,Northing,Elevation,GEDI02_A_slope_degrees_utm,rx_sample_count,cover_z_29,fhd_normal,pai_z_29,pavd_z_29
0,2019-07-23 02:29:57,BEAM1011,1,1,1.08,2.21,4.04,0.976120,1.137563,513688.964915,4.795170e+06,1815.427281,0.996506,755.0,0.0,1.417535,-0.0,-0.0
1,2019-07-23 02:29:57,BEAM1011,1,1,0.97,1.94,3.63,0.975038,1.137287,513736.570004,4.795201e+06,1815.428389,0.001113,749.0,0.0,1.340707,-0.0,-0.0
2,2019-07-23 02:29:57,BEAM1011,1,1,1.12,2.43,4.90,0.976903,1.137010,513784.205063,4.795233e+06,1814.398598,-1.034706,766.0,0.0,1.658798,-0.0,-0.0
3,2019-07-23 02:29:57,BEAM1011,1,1,0.93,1.91,3.59,0.971373,1.136734,513831.832065,4.795264e+06,1813.271707,-1.132363,745.0,0.0,1.269725,-0.0,-0.0
4,2019-07-23 02:29:57,BEAM1011,1,1,0.89,1.83,3.37,0.973743,1.136458,513879.447137,4.795295e+06,1813.272818,0.001117,745.0,0.0,0.790818,-0.0,-0.0


### Ground Truth Data

#### Loading Airborne Lidar Data

In [10]:
%%time
# path to your .laz file
laz_file_path = "/bsuhome/tnde/scratch/carbon_estimation/airborne_lidar/test_laz/USGS_LPC_ID_SouthernID_2018_D19_11TNH175700.laz"

# Load the .laz file
laz_data = laspy.read(laz_file_path)

# Print basic info about the data
print(laz_data)

<LasData(1.4, point fmt: <PointFormat(6, 0 bytes of extra dims)>, 26965964 points, 1 vlrs)>
CPU times: user 17.3 s, sys: 308 ms, total: 17.6 s
Wall time: 4.71 s


In [11]:
# List all available attributes in the .laz file
attributes = list(laz_data.point_format.dimension_names)
print("Attributes in the .laz file:", attributes)

Attributes in the .laz file: ['X', 'Y', 'Z', 'intensity', 'return_number', 'number_of_returns', 'synthetic', 'key_point', 'withheld', 'overlap', 'scanner_channel', 'scan_direction_flag', 'edge_of_flight_line', 'classification', 'user_data', 'scan_angle', 'point_source_id', 'gps_time']


In [12]:
# Assuming laz_data.classification is a pandas Series
cc = np.array(laz_data.classification)

# Find unique values in the classification array
unique_values = np.unique(cc)

# Print the unique values
print(unique_values)

[ 1  2  7 18]


In [13]:
# scan angle: close to nadir points are more reliable
# relative distributions of ground and not ground vegetation or elevation in each lidar tile
# check if segments with steeper slopes are more different.

#### Accessing the Data Attributes

In [14]:
# Access X, Y, Z coordinates
x_coords = laz_data.x
y_coords = laz_data.y
z_coords = laz_data.z

print(f"First 5 points - X: {x_coords[:5]}, Y: {y_coords[:5]}, Z: {z_coords[:5]}")
print("Length:", len(x_coords))

First 5 points - X: <ScaledArrayView([518808.3  518896.1  518814.31 518893.13 518894.76])>, Y: <ScaledArrayView([4770000.51 4770008.7  4770000.47 4770022.72 4770012.79])>, Z: <ScaledArrayView([1889.2  1891.7  1886.49 1890.03 1891.26])>
Length: 26965964


In [15]:
# Inspecting other attributes
print(f"Number of points: {len(laz_data)}")
print(f"Point returns: {laz_data.return_number}")

Number of points: 26965964
Point returns: <SubFieldView([1 1 2 ... 1 1 1])>


In [16]:
data_airbone_lidar = pd.DataFrame({
    "Easting": np.array(laz_data.x),
    "Northing": np.array(laz_data.y),
    "Elevation": np.array(laz_data.z),
    "intensity": laz_data.intensity,
    "return_number": np.array(laz_data.return_number),
    "classification": laz_data.classification,
    "user_data": laz_data.user_data,
    "point_source_id": laz_data.point_source_id,
    "gps_time": laz_data.gps_time,
    "scan_angle": laz_data.scan_angle,
    "number_of_returns": laz_data.number_of_returns
})
data_airbone_lidar = data_airbone_lidar[data_airbone_lidar["classification"].isin([1, 2, 3, 4, 5])]#.iloc[:1000000, :]
display(len(data_airbone_lidar["classification"].unique()))
display(len(data_airbone_lidar))
np.array(data_airbone_lidar["scan_angle"])

2

26807277

array([2666, 2833, 2666, ..., -500, -500, -500], dtype=int16)

In [17]:
display(data_airbone_lidar["scan_angle"].describe())

count    2.680728e+07
mean     5.388459e+01
std      2.138725e+03
min     -3.500000e+03
25%     -1.833000e+03
50%      1.660000e+02
75%      1.833000e+03
max      4.000000e+03
Name: scan_angle, dtype: float64

In [18]:
data_airbone_lidar = data_airbone_lidar[(data_airbone_lidar["scan_angle"] >= -3000) & (data_airbone_lidar["scan_angle"] <= 3000)]
display(data_airbone_lidar.head())
display(len(data_airbone_lidar))

,Easting,Northing,Elevation,intensity,return_number,classification,user_data,point_source_id,gps_time,scan_angle,number_of_returns
0,518808.30,4770000.51,1889.20,1392,1,1,0,3021,2.747243e+08,2666,"[2, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, ..."
1,518896.10,4770008.70,1891.70,5616,1,2,0,3021,2.747243e+08,2833,"[2, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, ..."
2,518814.31,4770000.47,1886.49,4160,2,1,0,3021,2.747243e+08,2666,"[2, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, ..."
3,518893.13,4770022.72,1890.03,5488,1,2,0,3021,2.747243e+08,2833,"[2, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, ..."
4,518894.76,4770012.79,1891.26,6144,1,1,0,3021,2.747243e+08,2833,"[2, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, ..."


22608211

In [19]:
# np.unique(laz_data.scan_angle)

In [20]:
# Calculate the bounding box
bounding_box = {
    "x_min": np.min(laz_data.x),  # Minimum X coordinate
    "x_max": np.max(laz_data.x),  # Maximum X coordinate
    "y_min": np.min(laz_data.y),  # Minimum Y coordinate
    "y_max": np.max(laz_data.y),  # Maximum Y coordinate
    "z_min": np.min(laz_data.z),  # Minimum Z coordinate
    "z_max": np.max(laz_data.z)   # Maximum Z coordinate
}

# Print the bounding box
print("Bounding Box Coordinates:")
print(f"X Min: {bounding_box['x_min']}, X Max: {bounding_box['x_max']}")
print(f"Y Min: {bounding_box['y_min']}, Y Max: {bounding_box['y_max']}")
print(f"Z Min: {bounding_box['z_min']}, Z Max: {bounding_box['z_max']}")

Bounding Box Coordinates:
X Min: 517500.0, X Max: 518999.99
Y Min: 4770000.0, Y Max: 4771499.99
Z Min: 1614.26, Z Max: 1934.1100000000001


In [21]:
np.unique(laz_data.scan_angle/100)

array([-35.  , -33.33, -31.66, -30.  , -28.33, -26.66, -25.  , -23.33,
       -21.66, -20.  , -18.33, -16.66, -15.  , -13.33, -11.66, -10.  ,
        -8.33,  -6.66,  -5.  ,  -3.33,  -1.66,   0.  ,   1.66,   3.33,
         5.  ,   6.66,   8.33,  10.  ,  11.66,  13.33,  15.  ,  16.66,
        18.33,  20.  ,  21.66,  23.33,  25.  ,  26.66,  28.33,  30.  ,
        31.66,  33.33,  35.  ,  36.66,  38.33,  40.  ])

#### Visualizing the Data

In [ ]:
# Create a 3D scatter plot
fig = plt.figure(figsize=(12, 10))
ax = fig.add_subplot(111, projection="3d")

# Scatter plot of the X, Y, Z coordinates
sc = ax.scatter(data_airbone_lidar["Easting"], 
                data_airbone_lidar["Northing"], 
                data_airbone_lidar["Elevation"], 
                c=data_airbone_lidar["Elevation"],  # Use elevation for color mapping
                cmap='viridis',  # Choose a colormap (viridis, plasma, etc.)
                s=0.1, alpha=0.5)

# Set labels
ax.set_xlabel("Easting", fontsize=15)
ax.set_ylabel("Northing", fontsize=15)
ax.set_zlabel("Elevation (m)", fontsize=15)
ax.tick_params(labelsize = 10)
plt.title("Lidar point cloud visualization")

# Add colorbar
cbar = plt.colorbar(sc)  # Associate the colorbar with the scatter plot
cbar.ax.tick_params(labelsize=12)  # set font size of colorbar tick labels
cbar.set_label("Elevation (m)", fontsize=15)  # Label for the colorbar

# Show the plot
plt.show()

In [ ]:
def plot_data_within_bounding_box(df1, df2, df3):
    """
    Finds the bounding box region of interest of the first DataFrame (df1), filters the second and third DataFrames (df2)
    for points within the bounding box, and plots the results.

    Parameters:
        df1 (DataFrame): The first DataFrame containing the points for defining the bounding box.
        df2 (DataFrame): The second DataFrame containing the points to be filtered by the bounding box.
        df3 (DataFrame): The second DataFrame containing the points to be filtered by the bounding box.

    Returns:
        None: Displays the plot.
    """

    # Find the bounding box region of interest from df1 (min/max of x and y)
    x_min, x_max = df1["Easting"].min(), df1["Easting"].max()
    y_min, y_max = df1["Northing"].min(), df1["Northing"].max()

    # Filter df2 to find all points within the bounding box of df1
    df2_filtered = df2[(df2["Easting"] >= x_min) & (df2["Easting"] <= x_max) &
                       (df2["Northing"] >= y_min) & (df2["Northing"] <= y_max)]
    
    # Filter df2 to find all points within the bounding box of df1
    df3_filtered = df3[(df3["Easting"] >= x_min) & (df3["Easting"] <= x_max) &
                       (df3["Northing"] >= y_min) & (df3["Northing"] <= y_max)]

    # Plot the data
    plt.figure(figsize=(10, 6))

    # Plot all points in df1 (whole dataset)
    plt.scatter(df1["Easting"], df1["Northing"], c="blue", alpha=0.05, label="DataFrame 1 (All points)", s=10)

    # Plot the selected points from df2 that fall within the bounding box
    plt.scatter(df2_filtered["Easting"], df2_filtered["Northing"], c="red", alpha=1.0, label="Filtered points from DataFrame 2", s=30)

    # Plot the selected points from df2 that fall within the bounding box
    plt.scatter(df3_filtered["Easting"], df3_filtered["Northing"], c="black", alpha=1.0, label="Filtered points from DataFrame 3", s=30)

    # Highlight the bounding box with a rectangle
    plt.plot([x_min, x_max, x_max, x_min, x_min], 
             [y_min, y_min, y_max, y_max, y_min], 
             color="green", linestyle="--", label="Bounding Box")

    # Labels and title
    plt.xlabel("X Coordinate")
    plt.ylabel("Y Coordinate")
    plt.title("GEDI and ICESat-2 Data within Bounding Box Region of Interest")
    plt.legend(fontsize=12)

    # Show the plot
    plt.show()

# Plot the points
plot_data_within_bounding_box(data_airbone_lidar, icesat2_gdf_atl08, gedi_data) 

In [ ]:
def filter_data(df1, df2, df3):
    """
    Filters footprints/segments from df2 and df3 within the bounding box of df1,
    drawing precise rectangles (13m x 100m) around each df2 filtered point, 
    and a circle of radius 12.5 meters around each df3 filtered point.

    Parameters:
        df1 (DataFrame): The reference dataset used to define the bounding region.
        df2 (DataFrame): The dataset whose points will be filtered and highlighted.
        df3 (DataFrame): The dataset whose points will be filtered and highlighted.

    Returns:
        DataFrames: Filtered subset of df2 and df3 within df1's bounding box.
    """

    # Bounding region of df1
    x_min, x_max = df1["Easting"].min(), df1["Easting"].max()
    y_min, y_max = df1["Northing"].min(), df1["Northing"].max()

    # Filter df2 within df1 bounding box
    df2_filtered = df2[
        (df2["Easting"] >= x_min) & (df2["Easting"] <= x_max) &
        (df2["Northing"] >= y_min) & (df2["Northing"] <= y_max)
    ]
    
    # Filter df3 within df1 bounding box
    df3_filtered = df3[
        (df3["Easting"] >= x_min) & (df3["Easting"] <= x_max) &
        (df3["Northing"] >= y_min) & (df3["Northing"] <= y_max)
    ]
    return df2_filtered, df3_filtered
icesat2_filtered, gedi_filtered = filter_data(data_airbone_lidar, icesat2_gdf_atl08, gedi_data)

In [ ]:
# icesat2_filtered.columns

In [ ]:
print(np.unique(icesat2_filtered["rgt"]))
print(np.unique(icesat2_filtered["gt"]))
list(np.unique(icesat2_filtered["date"]))[14:16]

In [ ]:
date_list = list(np.unique(icesat2_gdf_atl08["date"]))

# icesat2_filtered2 = icesat2_filtered[icesat2_filtered["date"] != "2023-10-21"]
# icesat2_filtered2 = icesat2_filtered[icesat2_filtered["date"].isin(date_list[28:30])]
icesat2_filtered2 = icesat2_filtered[icesat2_filtered["date"].isin(date_list[:10])]

# Plot the merged dataset
fig, ax = plt.subplots(figsize=(10, 10))
# Create a scatter plot where color is based on vegetation height values
scatter = ax.scatter(icesat2_filtered2.geometry.x, icesat2_filtered2.geometry.y,
                     c=icesat2_filtered2["h_mean_canopy"], cmap="viridis", s=50, alpha=1.0, label=" ")

# Add a color bar to the plot
cbar = plt.colorbar(scatter, ax=ax, orientation="vertical")
cbar.set_label("h_mean_canopy Values")

# Adding a basemap 
ctx.add_basemap(ax, crs=icesat2_gdf_atl08.crs, source=ctx.providers.Esri.WorldImagery)

# Formatting the map
plt.legend()
plt.title("Sample ICESat-2 Ground Tracks")
plt.xlabel("Easting (meters)")
plt.ylabel("Northing (meters)")
plt.show()


In [ ]:
def icesat2_gt_endflag(icesat2_df):
    """
    Logically find the ends of unique ICESat-2 ground tracks according to dates.
    
    Steps:
    - We sort data first by time and then ground track (gt) to make sure all the points are ordered correctly. 
    - Use the unique function to identify the first instance of unique pairs of date and gt in the dataset thus giving the start of a new track. 
    - Identify the row above each track start as the end of the previous track.
    
    Parameters:
    - icesat2_df: ICESat-2 dataframe, assumed to be a pandas DataFrame with 'time' (datetime64), 'gt' (beam), and 'norths' already defined.

    Returns:
    - end_flag: logical array indicating last point in each track. It is a binary array indicating where new tracks begin/end. 
      It is helps to avoid wrapping errors at track edges.
    """
    
    # Sort by 'time' and 'gt'
    sorted_df = icesat2_df.sort_values(by=["time", "gt"]).reset_index(drop=True)

    # Extract only the date (ignore time of day)
    dates = pd.to_datetime(sorted_df["time"].dt.date)

    # Create a combined key of date and 'gt' and find first occurrence 
    date_gt = pd.DataFrame({"date": dates, "gt": sorted_df["gt"]})
    unique_refs = date_gt.drop_duplicates(keep="first").index

    # Initialize end_flag with zeros (same length as number of rows)
    end_flag = np.zeros(len(sorted_df), dtype=int)

    # Flag unique_refs
    end_flag[unique_refs] = 1
    # Also flag the entry just before each unique_ref (unless it's 0)
    for idx in unique_refs:
        if idx > 0:
            end_flag[idx - 1] = 1
    # Flag the last element
    end_flag[-1] = 1
    return end_flag

end_flag = icesat2_gt_endflag(icesat2_filtered2)
end_flag

In [ ]:
def icesat2_footprint_corners(
    norths: np.ndarray,
    easts: np.ndarray,
    default_length: float,
    end_flag: np.ndarray
):
    """
    Calculate ICESat-2 footprint corners and orientation angles.
    This code inherently includes bearing-based orientation using the geometry of the ground track.

    Parameters:
    - norths: array of northing coordinates.
    - easts: array of easting coordinates.
    - default_length: default footprint length (e.g., 100 for ATLO8 or 40 meters for ATL06).
    - end_flag: logical array indicating last point in each track. It is a binary array indicating where new tracks begin/end. 
      It is helps to avoid wrapping errors at track edges.

    Returns:
    - xc: x corner coordinates (N x 6)
    - yc: y corner coordinates (N x 6)
    - theta: footprint orientation angles (N,). The angle theta between the icesat2 track and due east
    """
    footwidth = 13  # segment width (meters)
    n = norths.shape[0]

    theta = np.full((n, 2), np.nan)
    xc = np.full((n, 6), np.nan)
    yc = np.full((n, 6), np.nan)

    # create polygons of ICESat-2 footprints
    for r in range(n-1):
        # calculate the footprint orientation
        if r == 0 or end_flag[r] - end_flag[r - 1] == 0: # only angle pointed forwards
            theta[r, 0] = np.degrees(np.arctan2(norths[r + 1] - norths[r], easts[r + 1] - easts[r]))
            theta[r, 1] = theta[r, 0]
            footlength = default_length
        elif r == n - 1 or end_flag[r] - end_flag[r + 1] == 0: # only angle pointed backwards
            theta[r, 0] = np.degrees(np.arctan2(norths[r] - norths[r - 1], easts[r] - easts[r - 1]))
            theta[r, 1] = theta[r, 0]
            footlength = default_length
        else: # calculate angles in each direction
            theta[r, 0] = np.degrees(np.arctan2(norths[r] - norths[r - 1], easts[r] - easts[r - 1]))
            theta[r, 1] = np.degrees(np.arctan2(norths[r + 1] - norths[r], easts[r + 1] - easts[r]))
            footlength = 0.5 * np.sqrt((norths[r + 1] - norths[r - 1])**2 + (easts[r + 1] - easts[r - 1])**2)
            
        # find box edges along the RGT
        back_x = easts[r] - (footlength / 2) * np.cos(np.radians(theta[r, 0]))
        back_y = norths[r] - (footlength / 2) * np.sin(np.radians(theta[r, 0]))
        front_x = easts[r] + (footlength / 2) * np.cos(np.radians(theta[r, 1]))
        front_y = norths[r] + (footlength / 2) * np.sin(np.radians(theta[r, 1]))
        mean_theta = np.nanmean(theta[r, :])
        
        # find box edges perpendicular to the centroid
        xc[r, 0] = easts[r] + (footwidth / 2) * np.cos(np.radians(mean_theta + 90))
        yc[r, 0] = norths[r] + (footwidth / 2) * np.sin(np.radians(mean_theta + 90))
        xc[r, 1] = easts[r] + (footwidth / 2) * np.cos(np.radians(mean_theta - 90))
        yc[r, 1] = norths[r] + (footwidth / 2) * np.sin(np.radians(mean_theta - 90))
        
        # solve for corner coordinates
        xc[r, 2] = back_x + (footwidth / 2) * np.cos(np.radians(theta[r, 0] + 90))
        yc[r, 2] = back_y + (footwidth / 2) * np.sin(np.radians(theta[r, 0] + 90))
        xc[r, 3] = back_x + (footwidth / 2) * np.cos(np.radians(theta[r, 0] - 90))
        yc[r, 3] = back_y + (footwidth / 2) * np.sin(np.radians(theta[r, 0] - 90))

        xc[r, 4] = front_x + (footwidth / 2) * np.cos(np.radians(theta[r, 1] - 90))
        yc[r, 4] = front_y + (footwidth / 2) * np.sin(np.radians(theta[r, 1] - 90))
        xc[r, 5] = front_x + (footwidth / 2) * np.cos(np.radians(theta[r, 1] + 90))
        yc[r, 5] = front_y + (footwidth / 2) * np.sin(np.radians(theta[r, 1] + 90))
    return xc, yc, theta[:, 0]

In [ ]:
# Easting and Northing coordinates
norths_icesat2 = icesat2_filtered2["Northing"].values
easts_icesat2 = icesat2_filtered2["Easting"].values
norths_gedi = gedi_filtered["Northing"].values
easts_gedi = gedi_filtered["Easting"].values

# Call function to get segment corners
xc, yc, theta = icesat2_footprint_corners(norths = norths_icesat2, 
                                          easts = easts_icesat2, 
                                          default_length = 100.0, 
                                          end_flag = end_flag)

# Plot setup
fig, ax = plt.subplots(figsize=(10, 10))
patches1 = []
patches2 = []

x_min, x_max = data_airbone_lidar["Easting"].min(), data_airbone_lidar["Easting"].max()
y_min, y_max = data_airbone_lidar["Northing"].min(), data_airbone_lidar["Northing"].max()

# Bounding box for air borne lidar data.
ax.plot([x_min, x_max, x_max, x_min, x_min],
        [y_min, y_min, y_max, y_max, y_min],
        color="green", linestyle="--", linewidth=2, 
        label="Airborne lidar tile")

# ICESat-2 ATL08 segments: Iterate over segments to create polygon patches
for i in range(xc.shape[0]):
    if np.any(np.isnan(xc[i])) or np.any(np.isnan(yc[i])):
        continue  # Skip invalid segments
    
    # Create polygon from the 6 corners (can also simplify to first 4 if rectangle)
    footprint = [(xc[i, j], yc[i, j]) for j in range(6)]
    
    # You can use first 4 points to form a closed rectangle if desired:
    rect_points = [footprint[2], footprint[3], footprint[4], footprint[5]]

    polygon = MplPolygon(rect_points, closed=True)
    patches1.append(polygon)

# Add all patches to plot
p1 = PatchCollection(patches1, facecolor="skyblue", edgecolor="k", alpha=0.5)
ax.add_collection(p1)

# Plot segment centers
ax.plot(easts_icesat2, norths_icesat2, "ro", markersize=3, label="ICESat-2 segment centers")

# GEDI footprints: Draw a circle of radius 12.5 meters around each segment center
radius = 12.5  # meters
for x, y in zip(easts_gedi, norths_gedi):
    if np.isnan(x) or np.isnan(y):
        continue
    circle = Circle((x, y), radius=radius)
    patches2.append(circle)
    
# Add all patches to plot
p2 = PatchCollection(patches2, facecolor="magenta", edgecolor="k", alpha=0.5)
ax.add_collection(p2)

# Plot segment centers
ax.plot(easts_gedi, norths_gedi, "ko", markersize=3, label="GEDI footprint centers")

# Adding a basemap 
ctx.add_basemap(ax, crs=icesat2_gdf_atl08.crs, source=ctx.providers.Esri.WorldImagery)

# Axis settings
ax.set_aspect("equal")
ax.set_xlabel("Easting")
ax.set_ylabel("Northing")
# ax.set_title("GEDI Footprints and ICESat-2 ATL08 Segments")
ax.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

### Plot the unrotated vs rotated rectangles for comparison

In [ ]:
# Compute rotated corners
xc, yc, _ = icesat2_footprint_corners(norths_icesat2, easts_icesat2, 100, end_flag)

# Plot setup
fig, ax = plt.subplots(figsize=(10, 10))

# Bounding box for air borne lidar data.
ax.plot([x_min, x_max, x_max, x_min, x_min],
        [y_min, y_min, y_max, y_max, y_min],
        color="green", linestyle="--", linewidth=2, 
        label="Airborne lidar tile")

# Plot rotated rectangles
for i in range(xc.shape[0]):
    if np.any(np.isnan(xc[i])) or np.any(np.isnan(yc[i])):
        continue
    rotated_rect = [ (xc[i, j], yc[i, j]) for j in [2, 3, 4, 5] ]
    # ax.add_patch(Polygon(rotated_rect, closed=True, facecolor="none", edgecolor="green", label="Rotated" if i==0 else None))
    ax.add_patch(MplPolygon(rotated_rect, closed=True, facecolor="skyblue", alpha=0.6, edgecolor="green", label="Rotated" if i==0 else None))

# Plot unrotated rectangles
width, height = 13, 100
for i in range(len(easts_icesat2)):
    rect = [
        (easts_icesat2[i] - width / 2, norths_icesat2[i] - height / 2),
        (easts_icesat2[i] + width / 2, norths_icesat2[i] - height / 2),
        (easts_icesat2[i] + width / 2, norths_icesat2[i] + height / 2),
        (easts_icesat2[i] - width / 2, norths_icesat2[i] + height / 2),
    ]
    ax.add_patch(MplPolygon(rect, closed=True, facecolor="none", edgecolor="red", linestyle="--", linewidth=1.2, label="Unrotated" if i==0 else None))
    # ax.add_patch(MplPolygon(rect, closed=True, facecolor="red", alpha=0.2, edgecolor="red", linestyle="--", label="Unrotated" if i==0 else None))

# Centers
ax.plot(easts_icesat2, norths_icesat2, "ko", markersize=3, label="Centers")


# Plot segment centers
ax.plot(easts_icesat2, norths_icesat2, "ro", markersize=3, label="ICESat-2 segment centers")

###############################
# GEDI footprints: Draw a circle of radius 12.5 meters around each segment center
patches2 = []
# GEDI footprints: Draw a circle of radius 12.5 meters around each segment center
radius = 12.5  # meters
for x, y in zip(easts_gedi, norths_gedi):
    if np.isnan(x) or np.isnan(y):
        continue
    circle = Circle((x, y), radius=radius)
    patches2.append(circle)
    
# Add all patches to plot
p2 = PatchCollection(patches2, facecolor="magenta", edgecolor="k", alpha=0.5)
ax.add_collection(p2)

# Plot segment centers
ax.plot(easts_gedi, norths_gedi, "ko", markersize=3, label="GEDI footprint centers")

#############################
# Adding a basemap
ctx.add_basemap(ax, crs=icesat2_gdf_atl08.crs, source=ctx.providers.Esri.WorldImagery)

# Legend & final touches
ax.set_aspect("equal")
ax.set_xlabel("Easting")
ax.set_ylabel("Northing")
ax.set_title("Rotated vs Unrotated ICESat-2 Segments")
ax.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

### Color segments by date

In [ ]:
# ICESat-2 ATL08 Segments
# Compute corners
xc, yc, _ = icesat2_footprint_corners(norths_icesat2, easts_icesat2, 100, end_flag)

# Convert dates to strings
# 1. Sort by 'time' and 'gt'
sorted_icesat2_df = icesat2_filtered2.sort_values(by=["time", "gt"]).reset_index(drop=True)
# 2. Extract only the date (ignore time of day)
dates_icesat2 = pd.to_datetime(sorted_icesat2_df["time"].dt.date)
date_strings_icesat2 = pd.to_datetime(dates_icesat2).dt.strftime('%Y-%m-%d')

# Get unique dates and assign each a color
unique_dates_icesat2 = sorted(set(date_strings_icesat2))
colors_icesat2 = cm.get_cmap("tab10", len(unique_dates_icesat2))
date_color_map_icesat2 = {date: colors_icesat2(i) for i, date in enumerate(unique_dates_icesat2)}


# GEDI Footprints
# 1. Sort by 'date' and 'BEAM' and convert dates to strings
sorted_gedi_df = gedi_filtered.sort_values(by=["date", "BEAM"]).reset_index(drop=True)
# 2. Extract only the date (ignore time of day)
dates_gedi = pd.to_datetime(sorted_gedi_df["date"].dt.date)
date_strings_gedi = pd.to_datetime(dates_gedi).dt.strftime('%Y-%m-%d')

# Get unique dates and assign each a color
unique_dates_gedi = sorted(set(date_strings_gedi))
colors_gedi = cm.get_cmap("tab10", len(unique_dates_gedi))
date_color_map_gedi = {date: colors_gedi(i) for i, date in enumerate(unique_dates_gedi)}


# Plot segments
fig, ax = plt.subplots(figsize=(12, 10))

# Bounding box for air borne lidar data.
ax.plot([x_min, x_max, x_max, x_min, x_min],
        [y_min, y_min, y_max, y_max, y_min],
        color="green", linestyle="--", linewidth=2, 
        label="Airborne lidar tile")

# ICESat-2 plot
for i in range(len(norths_icesat2)):
    polygon_x = list(xc[i, [2, 3, 4, 5, 2]]) # closed loop
    polygon_y = list(yc[i, [2, 3, 4, 5, 2]])
    date_str_icesat2 = date_strings_icesat2[i]
    ax.plot(polygon_x, polygon_y, color=date_color_map_icesat2[date_str_icesat2], linewidth=1)
    # Plot segment centers
    ax.plot(easts_icesat2, norths_icesat2, "ro", markersize=3, label="Segment centers")

# ICESat-2 legend handles
handles_icesat2 = [
    plt.Line2D([0], [0], color=color, lw=2, label=f"ICESat2: {label}")
    for label, color in date_color_map_icesat2.items()
]


# GEDI plot: Plot circular segments
for i in range(len(norths_gedi)):
    center_x, center_y = easts_gedi[i], norths_gedi[i]
    date_str_gedi = date_strings_gedi[i]
    circle = plt.Circle((center_x, center_y), 12.5, 
                        color=date_color_map_gedi[date_str_gedi], 
                        fill=False, linewidth=1)
    ax.add_patch(circle)

# Plot segment centers
ax.plot(easts_gedi, norths_gedi, "ko", markersize=3, label="Segment centers")

# GEDI legend handles
handles_gedi = [
    plt.Line2D([0], [0], color=color, lw=2, label=f"GEDI: {label}")
    for label, color in date_color_map_gedi.items()
]

# Combine both handles and show a single legend
combined_handles = handles_icesat2 + handles_gedi
ax.legend(handles=combined_handles, title="Observation Dates", bbox_to_anchor=(1.05, 1), loc="upper left")

# Adding a basemap
ctx.add_basemap(ax, crs=icesat2_gdf_atl08.crs, source=ctx.providers.Esri.WorldImagery)

ax.set_title("GEDI Footprints and ICESat-2 Segments and GEDI Footprints Colored by Date")
ax.set_xlabel("Easting")
ax.set_ylabel("Northing")
ax.axis("equal")
plt.tight_layout()
plt.show()

### Color segments by Ground Tracks

In [ ]:
# ICESat-2: Unique beam identifiers
beams_icesat2 = icesat2_filtered2["gt"].values
unique_beams_icesat2 = sorted(set(beams_icesat2))  # Example: ['gt1l', 'gt1r', 'gt2l', 'gt2r', 'gt3l', 'gt3r']
beam_to_color_icesat2 = {beam: get_cmap('tab10')(i) for i, beam in enumerate(unique_beams_icesat2)}

# GEDI: Unique beam identifiers
beams_gedi = gedi_filtered["BEAM"].values
unique_beams_gedi = sorted(set(beams_gedi))  # Example: ['gt1l', 'gt1r', 'gt2l', 'gt2r', 'gt3l', 'gt3r']
beam_to_color_gedi = {beam: get_cmap('tab10')(i) for i, beam in enumerate(unique_beams_gedi)}


fig, ax = plt.subplots(figsize=(12, 10))

# Bounding box for air borne lidar data.
ax.plot([x_min, x_max, x_max, x_min, x_min],
        [y_min, y_min, y_max, y_max, y_min],
        color="green", linestyle="--", linewidth=2, 
        label="Airborne lidar tile")

# ICESat-2 plot
for i in range(len(xc)):
    corners = [(xc[i, j], yc[i, j]) for j in [2, 3, 4, 5]]  # segment edges
    beam_id_icesat2 = beams_icesat2[i]
    color_icesat2 = beam_to_color_icesat2.get(beam_id_icesat2, "gray")

    polygon = MplPolygon(corners, closed=True, facecolor=color_icesat2, edgecolor="k", alpha=0.6, label=beam_id_icesat2 if i == 0 else "")
    ax.add_patch(polygon)
    # Plot segment centers
    ax.plot(easts_icesat2, norths_icesat2, "ro", markersize=2, label="Segment centers")

# GEDI legend handles
handles_icesat2 = [Patch(color=beam_to_color_icesat2[b], label=f"ICESat-2: {b}") for b in unique_beams_icesat2]

# GEDI plot
# Draw circular segments
for i in range(len(norths_gedi)):
    beam_id_gedi = beams_gedi[i]
    color_gedi = beam_to_color_gedi.get(beam_id_gedi, "gray")

    # Center of footprint
    cx, cy = easts_gedi[i], norths_gedi[i]

    # Circle instead of rectangle
    circle = Circle((cx, cy), radius=12.5, facecolor=color_gedi, edgecolor="k", alpha=0.6, label=beam_id_gedi if i == 0 else "")
    ax.add_patch(circle)

# Plot segment centers
ax.plot(easts_gedi, norths_gedi, "ro", markersize=2, label="Footprint centers")

# GEDI legend handles
handles_gedi = [Patch(color=beam_to_color_gedi[b], label=f"GEDI: {b}") for b in unique_beams_gedi]

# Combined handles
combined_handles = handles_icesat2 + handles_gedi

# legend
ax.legend(handles=combined_handles, title="Beam Tracks", bbox_to_anchor=(1.05, 1), loc="upper left")

# Adding a basemap (Optional)
ctx.add_basemap(ax, crs=icesat2_gdf_atl08.crs, source=ctx.providers.Esri.WorldImagery)

ax.set_xlabel("Easting")
ax.set_ylabel("Northing")
ax.set_title("GEDI Footprints and ICESat-2 Segments Colored by Ground Track")
plt.axis("equal")
plt.tight_layout()
plt.show()

### Classify canopies in ICESat-2 segments

In [ ]:
utm_code = "epsg:6340"
def classify_data_footprints(df1, df2, df3, xc, yc, radius=12.5):
    """
    Classify each ICESat-2 segment (df1) and GEDI footprint (df2) by checking which df3 points fall within
    the ICESat-2 rotated polygons and GEDI circular footprints, respectively. Assign canopy class based on the majority of vegetation points.

    Parameters:
    - df1 (DataFrame): DataFrame with ICESat-2 segments.
    - df2 (DataFrame): DataFrame with GEDI footprints.
    - df3 (DataFrame): Lidar point cloud with vegetation classifications.
    - xc (ndarray): ICESat-2 x-coordinates of rotated polygon corners (N x 4 or 6).
    - yc (ndarray): ICESat-2 y-coordinates of rotated polygon corners (N x 4 or 6).
    - radius (float): Radius of the circular region around each GEDI footprint center (in meters).

    Returns:
    - df1 (DataFrame): Updated with "assigned_class" column.
    - df2 (DataFrame): Updated with "assigned_class" column.
    """
    # GeoDataFrame for df3 (lidar points)
    gdf3 = gpd.GeoDataFrame(
        df3,
        geometry=gpd.points_from_xy(df3["Easting"], df3["Northing"]),
        crs=utm_code
    )

    # ICESat-2 data
    # polygons from segment corners (only valid ones)
    polygons = []
    segment_indices = []
    for i in range(df1.shape[0]):
        if not np.any(np.isnan(xc[i])) and not np.any(np.isnan(yc[i])):
            poly = Polygon([(xc[i, j], yc[i, j]) for j in [2, 3, 4, 5]])
            polygons.append(poly)
            segment_indices.append(i)

    gdf1 = gpd.GeoDataFrame({
        "segment_index": segment_indices,
        "geometry": polygons
    }, crs=utm_code)

    # Spatial join
    joined_df1 = gpd.sjoin(gdf3, gdf1, how="inner", predicate="within")

    # Filter and compute dominant class per segment
    veg_classes_df1 = joined_df1[joined_df1["classification"].isin([1, 2, 3, 4, 5])]
    class_mode_df1 = veg_classes_df1.groupby("segment_index")["classification"].agg(lambda x: x.value_counts().idxmax())

    # Initialize all to 2 and assign based on matches
    assigned_class_df1 = np.full(len(df1), 2)
    for idx, val in class_mode_df1.items():
        assigned_class_df1[idx] = val

    # Assign result to df1
    df1["assigned_class"] = assigned_class_df1
    
    ########################################################
    ########################################################
    # GEDI data
    # Create circular buffers around each segment center in df1
    centers_df2 = gpd.points_from_xy(df2["Easting"], df2["Northing"])
    gdf2 = gpd.GeoDataFrame({
        "footprint_index": df2.index, 
        "geometry": centers_df2.buffer(radius)
    }, crs=utm_code)

    # Spatial join
    joined_df2 = gpd.sjoin(gdf3, gdf2, how="inner", predicate="within")

    # Filter and compute dominant class per segment
    veg_classes_df2 = joined_df2[joined_df2["classification"].isin([1, 2, 3, 4, 5])]
    class_mode_df2 = veg_classes_df2.groupby("footprint_index")["classification"].agg(lambda x: x.value_counts().idxmax())

    # Map segment_index to actual row positions in df1
    footprint_index_to_pos_df2 = {foot_idx: pos for pos, foot_idx in enumerate(df2.index)}

    # Set default class as 2
    assigned_class_df2 = np.full(len(df2), 2)
    
    # Assign values safely using position
    for foot_idx, val in class_mode_df2.items():
        if foot_idx in footprint_index_to_pos_df2:
            assigned_class_df2[footprint_index_to_pos_df2[foot_idx]] = val

    # Add result to df1
    df2["assigned_class"] = assigned_class_df2
    
    return df1, df2

# Classify the points in segments based on the bounding boxes and points in df2
icesat2_classified, gedi_classified = classify_data_footprints(icesat2_filtered2, gedi_filtered, data_airbone_lidar, xc, yc)
# Display the result
display(icesat2_classified.head())
display(gedi_classified.head())

In [ ]:
# color map for classes
class_colors = {
    1: "green",   # # unclassified but it is vegetation in our case
    3: "orange",  # low vegetation
    4: "red",     # medium vegetation
    5: "purple",  # high vegetation
    2: "brown"     # default ground 
}

fig, ax = plt.subplots(figsize=(10, 10))

# Bounding box for air borne lidar data.
ax.plot([x_min, x_max, x_max, x_min, x_min],
        [y_min, y_min, y_max, y_max, y_min],
        color="green", linestyle="--", linewidth=2, 
        label="Airborne lidar tile")


# GEDI data
patches1 = []
colors1 = []
# Plotting each polygon segment
for i in range(icesat2_classified.shape[0]):
    if np.any(np.isnan(xc[i])) or np.any(np.isnan(yc[i])):
        continue

    rect = [(xc[i, j], yc[i, j]) for j in [2, 3, 4, 5]]
    polygon = MplPolygon(rect, closed=True)
    patches1.append(polygon)

    assigned_class_icesat2 = icesat2_classified.iloc[i]["assigned_class"]
    colors1.append(class_colors.get(assigned_class_icesat2, "brown"))

# Add patch collection
# p = PatchCollection(patches, facecolor=colors, edgecolor='k', linewidths=1.5, alpha=0.8)
p1 = PatchCollection(patches1, facecolor="none", edgecolor=colors1, linewidths=1.5, alpha=0.8)
ax.add_collection(p1)

# Plotting segment centers
ax.plot(icesat2_classified["Easting"], icesat2_classified["Northing"], "ko", markersize=2, label="Segment Centers")

# Axis and Legend
legend_labels_icesat2 = [plt.Line2D([0], [0], marker='s', color='w', 
                                    label=f"Class {cls}", markersize=10, 
                                    markerfacecolor=color) for cls, color in class_colors.items()]
# ax.legend(handles=legend_labels_icesat2, title="Assigned Canopy Class")

###########################################################################
###########################################################################
# GEDI data
patches2 = []
colors2 = []

# Plotting each circular segment
for i in range(gedi_classified.shape[0]):
    easting_gedi = gedi_classified.iloc[i]["Easting"]
    northing_gedi = gedi_classified.iloc[i]["Northing"]

    # Skip if coordinates are missing
    if np.isnan(easting_gedi) or np.isnan(northing_gedi):
        continue

    # Make circular buffer (radius = 12.5 meters)
    circle = Circle((easting_gedi, northing_gedi), radius=12.5)
    patches2.append(circle)

    assigned_class_gedi = gedi_classified.iloc[i]["assigned_class"]
    colors2.append(class_colors.get(assigned_class_gedi, "brown"))

# Add patch collection
p2 = PatchCollection(patches2, facecolor="none", edgecolor=colors2, linewidths=1.5, alpha=0.8)
ax.add_collection(p2)

# Plot segment centers
ax.plot(gedi_classified["Easting"], gedi_classified["Northing"], "ko", markersize=2, label="Segment Centers")

# Axis and legend
legend_labels_gedi = [plt.Line2D([0], [0], marker='s', color='w', 
                                 label=f"Class {cls}", markersize=10, 
                                 markerfacecolor=color) for cls, color in class_colors.items()]
ax.legend(handles=legend_labels_gedi, title="Assigned Canopy Class")

###########################################################
###########################################################
# # Combine both legend handles
# legend_labels_combined = [
#     plt.Line2D([0], [0], marker='s', color='w', label=f"ICESat-2 Class {cls}", 
#                markersize=10, markerfacecolor=color) for cls, color in class_colors.items()] + [plt.Line2D([0], [0], marker='s', color='w', label=f"GEDI Class {cls}", 
#                                                                                                            markersize=10, markerfacecolor=color) for cls, color in class_colors.items()]

# # Plot the combined legend
# ax.legend(handles=legend_labels_combined, title="Assigned Canopy Class", loc="upper right")

# Adding a basemap
ctx.add_basemap(ax, crs=icesat2_gdf_atl08.crs, source=ctx.providers.Esri.WorldImagery)

ax.set_xlabel("Easting")
ax.set_ylabel("Northing")
ax.set_title("GEDI Footprints and ICESat-2 Segments Colored by Assigned Canopy Class")
ax.set_aspect("equal")
plt.grid(True)
plt.tight_layout()
plt.show()

In [ ]:
class_colors = {
    2: ("Ground", "brown"),  # default
    1: ("Vegetation", "green"), # unclassified but it is vegetation only in our specific case
    3: ("Low Vegetation", "orange"),
    4: ("Medium Vegetation", "black"),
    5: ("High Vegetation", "purple"),
}

# ICESat-2 data
# reset the index 
icesat2_classified = icesat2_classified.reset_index(drop=True)
assert len(icesat2_classified) == xc.shape[0], "Mismatch between df1 rows and segment coordinates" # safety check for mismatched sizes

# GeoDataFrame for rotated segment polygons
polygons = []
for i in range(icesat2_classified.shape[0]):
    if np.any(np.isnan(xc[i])) or np.any(np.isnan(yc[i])):
        polygons.append(None)
        continue
    rect = [(xc[i, j], yc[i, j]) for j in [2, 3, 4, 5]]
    polygons.append(Polygon(rect))

icesat2_segments = gpd.GeoDataFrame(icesat2_classified.copy(), geometry=polygons, crs=utm_code)  # use correct CRS

# GeoDataFrame for lidar points
gdf_points = gpd.GeoDataFrame(
    data_airbone_lidar.copy(),
    geometry=[Point(xy) for xy in zip(data_airbone_lidar["Easting"], data_airbone_lidar["Northing"])],
    crs=utm_code
)

# Spatial join: points inside any segment
gdf_joined_icesat2 = gpd.sjoin(gdf_points, icesat2_segments, predicate="within", how="inner")

# 4. Plotting segments and lidar points within them
fig, ax = plt.subplots(figsize=(10, 10))

# Bounding box for air borne lidar data.
ax.plot([x_min, x_max, x_max, x_min, x_min],
        [y_min, y_min, y_max, y_max, y_min],
        color="green", linestyle="--", linewidth=2, 
        label="Airborne lidar tile")

# Plotting ICESat-2 segments
patches1 = []
segment_colors = []
for i in range(len(icesat2_segments)):
    if icesat2_segments.geometry.iloc[i] is None:
        continue
    polygon = MplPolygon(icesat2_segments.geometry.iloc[i].exterior.coords, closed=True)
    patches1.append(polygon)
    c = class_colors.get(icesat2_segments.iloc[i]["assigned_class"], ("Unknown", "gray"))[1]
    segment_colors.append(c)

# segment_collection = PatchCollection(patches, facecolor=segment_colors, edgecolor='k', linewidths=1.5, alpha=0.8)
segment_collection = PatchCollection(patches1, facecolor="none", edgecolor=segment_colors, linewidths=1.5, alpha=0.8)
ax.add_collection(segment_collection)

# Plotting segment centers
ax.plot(icesat2_classified["Easting"], icesat2_classified["Northing"], "ko", markersize=2, label="Segment Centers")

# Plot lidar points (filtered)
for class_id, (class_name, color) in class_colors.items():
    subset = gdf_joined_icesat2[gdf_joined_icesat2["classification"] == class_id]
    ax.scatter(subset.geometry.x, subset.geometry.y, label=class_name, color=color, s=2, alpha=0.8)

    
# GEDI data
# Reset index for safety
gedi_classified = gedi_classified.reset_index(drop=True)
assert len(gedi_classified) == gedi_classified.shape[0], "Mismatch between df1 rows and segment coordinates"

# GeoDataFrame with circular buffers (radius = 12.5 meters)
circle_geometries = []
for i in range(gedi_classified.shape[0]):
    if np.isnan(gedi_classified.iloc[i]["Easting"]) or np.isnan(gedi_classified.iloc[i]["Northing"]):
        circle_geometries.append(None)
    else:
        center = Point(gedi_classified.iloc[i]["Easting"], gedi_classified.iloc[i]["Northing"])
        buffer = center.buffer(12.5)  # radius in meters
        circle_geometries.append(buffer)

gedi_footprints = gpd.GeoDataFrame(gedi_classified.copy(), geometry=circle_geometries, crs=utm_code)

# Spatial join: points inside any circular buffer
gdf_joined_gedi = gpd.sjoin(gdf_points, gedi_footprints, predicate="within", how="inner")

# Plotting ICESat-2 circular buffers
patches2 = []
footprint_colors = []
for i in range(len(gedi_footprints)):
    if gedi_footprints.geometry.iloc[i] is None:
        continue
    centroid = gedi_footprints.geometry.iloc[i].centroid
    circle_patch = Circle((centroid.x, centroid.y), radius=12.5)
    patches2.append(circle_patch)
    c = class_colors.get(gedi_footprints.iloc[i]["assigned_class"], ("Unknown", "gray"))[1]
    footprint_colors.append(c)

footprint_collection = PatchCollection(patches2, facecolor="none", edgecolor=footprint_colors, linewidths=1.5, alpha=0.8)
ax.add_collection(footprint_collection)

# Plotting segment centers
ax.plot(gedi_classified["Easting"], gedi_classified["Northing"], "ro", markersize=2, label="Footprint Centers")

# Plotting lidar points (filtered)
for class_id, (class_name, color) in class_colors.items():
    subset = gdf_joined_gedi[gdf_joined_gedi["classification"] == class_id]
    # ax.scatter(subset.geometry.x, subset.geometry.y, label=class_name, color=color, s=2, alpha=0.8)
    ax.scatter(subset.geometry.x, subset.geometry.y, color=color, s=2, alpha=0.8)

# Adding a basemap
ctx.add_basemap(ax, crs=icesat2_gdf_atl08.crs, source=ctx.providers.Esri.WorldImagery)    

ax.set_aspect("equal")
ax.set_xlabel("Easting")
ax.set_ylabel("Northing")
ax.set_title("Lidar Points Within Classified GEDI Footprints and ICESat-2 Segments")
ax.legend(loc="upper right", fontsize=9)
plt.grid(True)
plt.tight_layout()
plt.show()

# The End!!!